# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [31]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import confusion_matrix
import pickle


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [32]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}

In [33]:
# load data from database
def load_data():
    engine = create_engine('sqlite:///DisasterResponse123.db')
    df = pd.read_sql_table('DisasterResponse123', engine)
    X = df["message"]
    Y = df.drop(['message', 'genre', 'id', 'original'], axis = 1)
    return X,Y

In [34]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [35]:
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip() 
        clean_tokens.append(clean_tok)

    return clean_tokens

In [36]:
X,Y=load_data()
for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n') 

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', '-', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80-90', 'destroyed', '.', 'only', 'hospital', 'st.', 'croix', 'functioning', '.', 'needs', 'supply', 'desperately', '.'] 

says: west side of Haiti, rest of the country today and tonight
['say', ':', 'west', 'side', 'of', 'haiti', ',', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [37]:
 pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [38]:
#split data in train-test 
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.3, random_state = 42)


In [39]:
print(f'X_train:{X_train.shape}')
print(f'y_train:{y_train.shape}')
print(f'X_test:{X_test.shape}')
print(f'y_test:{y_test.shape}')

X_train:(18351,)
y_train:(18351, 36)
X_test:(7865,)
y_test:(7865, 36)


In [40]:
#Train classifier
pipeline_fit=pipeline.fit(X_train, y_train)
pipeline_fit

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [41]:
def acc_report(model, X_test, y_test):
    '''
    #https://stackoverflow.com/questions/45346550/valueerror-unknown-label-type-unknown
   Function to generate classification report
   '''
    y_pred = model.predict(X_test)
    for i, x in enumerate(y_test):
        print(x)
        print(classification_report(y_test[x], y_pred[:, i]))

In [42]:
acc_report(pipeline, X_test, y_test)

related
             precision    recall  f1-score   support

          0       0.60      0.34      0.44      1873
          1       0.81      0.93      0.87      5934
          2       0.50      0.09      0.15        58

avg / total       0.76      0.78      0.76      7865

request
             precision    recall  f1-score   support

          0       0.89      0.99      0.93      6533
          1       0.84      0.38      0.52      1332

avg / total       0.88      0.88      0.86      7865

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7829
          1       0.00      0.00      0.00        36

avg / total       0.99      1.00      0.99      7865

aid_related
             precision    recall  f1-score   support

          0       0.72      0.88      0.79      4646
          1       0.74      0.51      0.60      3219

avg / total       0.73      0.73      0.71      7865

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [43]:
pipeline.score(X_test, y_test)

0.21869040050858232

### 6. Improve your model
Use grid search to find better parameters. 

In [44]:
#number of trees in Random Forest
n_estimators=[int(x) for x in np.linspace(start=10, stop=80,num=10)]
#numner of features to consider at every split
max_features=['auto','sqrt']
#maximum number of levels in a tree
max_depth=[2,4]
#minimum numbers of samples required to split a node
min_samples_split=[2,5]
#minimum numbers of samples required at each leaf node
min_samples_leaf=[1,2]
#method of selecting sample for training each tree
bootstrap=[True,False]

In [46]:
param_grid={'n_estimators':n_estimators,
               'max_features':max_features,
               'max_depth':max_depth,
               'min_samples_split':min_samples_split,
                'min_samples_leaf':min_samples_leaf,
                'bootstrap':bootstrap
               }
print(param_grid)

{'n_estimators': [10, 17, 25, 33, 41, 48, 56, 64, 72, 80], 'max_features': ['auto', 'sqrt'], 'max_depth': [2, 4], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]}


In [47]:
rf_Model=RandomForestClassifier
rf_Grid=GridSearchCV(estimator=rf_Model,param_grid=param_grid,cv=3,verbose=3,n_jobs=4)

In [48]:
rf_Grid=(X_train,y_train)

In [49]:
rf_Grid.best_params_

AttributeError: 'tuple' object has no attribute 'best_params_'

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.